# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cd_weather = pd.read_csv("../weatherpy/city_weather.csv")
cd_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,72.050,79.0,0.0,4.85,PF,08-18-2020
1,stokmarknes,68.56,14.91,51.998,85.0,92.0,2.24,NO,08-18-2020
2,mataura,-46.19,168.86,43.844,81.0,100.0,2.27,NZ,08-18-2020
3,najran,17.49,44.13,93.200,17.0,40.0,2.60,SA,08-18-2020
4,kapaa,22.08,-159.32,82.400,65.0,20.0,7.20,US,08-18-2020
...,...,...,...,...,...,...,...,...,...
549,linkou,45.30,130.28,65.012,94.0,100.0,1.55,CN,08-18-2020
550,helong,42.54,129.00,71.636,75.0,11.0,3.11,CN,08-18-2020
551,abnub,27.27,31.15,87.800,43.0,0.0,4.60,EG,08-18-2020
552,aleksandriyskaya,44.23,43.35,68.000,45.0,0.0,1.00,RU,08-18-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Define Latitude & Longitude as Location
locations = cd_weather[['Lat', 'Lng']].astype(float)

# Humidity as float for weight
humidity = cd_weather['Humidity'].astype(float)

In [5]:
# Create a poverty Heatmap layer
# from https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2) 

heat_layer = gmaps.heatmap_layer(
            locations, 
            weights=humidity, 
            max_intensity=100,         
            dissipating=False, 
            point_radius = 2.5, 
            opacity=1,
            gradient=None
                                ) # Formatting options found by shift tab

fig.add_layer(heat_layer)

plt.savefig("humidityheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

At the time and date of the query, there tended to be higher pockets of humidity near the coastlines or island chains, across the globe.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.
#A max temperature lower than 80 degrees but higher than 70.
sub_80 = cd_weather.loc[cd_weather["Max Temp"]<80]
sub_80_70_plus = sub_80.loc[cd_weather["Max Temp"]>70]
sub_80_70_plus_wind = sub_80_70_plus.loc[cd_weather["Wind Speed"]<10]
ideal_weather = sub_80_70_plus_wind.loc[cd_weather["Cloudiness"]==0]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,72.050,79.0,0.0,4.85,PF,08-18-2020
11,lata,40.16,-8.33,75.992,57.0,0.0,1.34,PT,08-18-2020
40,ponta do sol,32.67,-17.10,73.994,90.0,0.0,1.35,PT,08-18-2020
124,faanui,-16.48,-151.75,78.296,78.0,0.0,8.20,PF,08-18-2020
197,taybad,34.74,60.78,70.070,33.0,0.0,5.76,IR,08-18-2020
236,reconquista,-29.15,-59.65,73.850,28.0,0.0,2.43,AR,08-18-2020
263,kropotkin,45.44,40.58,70.196,37.0,0.0,1.73,RU,08-18-2020
287,kuryk,43.20,51.65,72.212,61.0,0.0,2.52,KZ,08-18-2020
399,luau,-10.71,22.22,70.880,23.0,0.0,1.46,AO,08-18-2020
445,mugla,37.23,28.36,77.000,64.0,0.0,1.50,TR,08-18-2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = ""
hotel_df.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
453,maldonado,-34.67,-54.92,71.600,37.0,0.0,4.60,UY,08-18-2020,
478,bahar,34.91,48.44,75.200,36.0,0.0,5.10,IR,08-18-2020,
511,javanrud,34.80,46.52,78.800,21.0,0.0,2.05,IR,08-18-2020,
543,young,-32.68,-57.63,73.004,25.0,0.0,1.79,UY,08-18-2020,
553,kijang,35.24,129.21,78.800,88.0,0.0,0.50,KR,08-18-2020,


In [8]:
params = {    
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [10]:
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (KeyError, IndexError):
        print("Hotel not found. Skipping...")

In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rikitea,-23.12,-134.97,72.050,79.0,0.0,4.85,PF,08-18-2020,Pension Maro'i
11,lata,40.16,-8.33,75.992,57.0,0.0,1.34,PT,08-18-2020,Casa Maquia
40,ponta do sol,32.67,-17.10,73.994,90.0,0.0,1.35,PT,08-18-2020,Hotel do Campo
124,faanui,-16.48,-151.75,78.296,78.0,0.0,8.20,PF,08-18-2020,Bora-Bora Pearl Beach Resort & Spa
197,taybad,34.74,60.78,70.070,33.0,0.0,5.76,IR,08-18-2020,لی پوش jean center
236,reconquista,-29.15,-59.65,73.850,28.0,0.0,2.43,AR,08-18-2020,Grand Hotel
263,kropotkin,45.44,40.58,70.196,37.0,0.0,1.73,RU,08-18-2020,Hotel Europe Kropotkin
287,kuryk,43.20,51.65,72.212,61.0,0.0,2.52,KZ,08-18-2020,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...
399,luau,-10.71,22.22,70.880,23.0,0.0,1.46,AO,08-18-2020,Kimbo Hotel
445,mugla,37.23,28.36,77.000,64.0,0.0,1.50,TR,08-18-2020,Petek Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

There are ideal weather conditions throughout the world, with 2 in the South Pacific, 3 in South America, 2 in Western Europe, 1 in Africa, 3 in the middle